Output URI
s3://groups-bucket-dblab-905418150721/group23/outputs/query1/

Να ταξινομημηθούν, σε φθίνουσα σειρά, οι ηλικιακές ομάδες των θυμάτων σε περιστατικά που περιλαμβάνουν οποιαδήποτε μορφή “βαριάς σωματικής βλάβης”. Θεωρείστε τις εξής ηλικιακές ομάδες:\
• Παιδιά: < 18 • Νεαροί ενήλικοι: 18 – 24
• Ενήλικοι: 25 – 64 • Ηλικιωμένοι: >64

Να υλοποιηθεί το Query 1 χρησιμοποιώντας τα DataFrame και RDD APIs. Να εκτελέσετε και
τις δύο υλοποιήσεις με 4 Spark executors. Υπάρχει διαφορά στην επίδοση μεταξύ των δύο APIs;\
Αιτιολογήσετε την απάντησή σας.

Αρχικά θέτουμε τον configuration όπως το θέλει η εκφώνηση:

In [1]:
%%configure -f
{
    "conf": {
        "spark.executor.instances": "4",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "2",
        "spark.driver.memory": "4g"
    }
}


Για να υλοποιήσουμε το query με RDD θα κάνουμε τις εξής πράξεις:
* Ανοίγουμε τα 2 csv
* Κάνουμε union στα 2 csv
* κάνουμε map στη συνάρτηση parse_csv (θέλουμε να αγνοούνται τα "," μέσα σε strings γι αυτό δεν μπορούμε να κάνουμε split στο ",")
* κρατάμε μόνο τις γραμμές που έχουν τον όρο "aggravated assault" στην σωστή στήλη (10)
* για κάθε μια από τις γραμμές κρατάμε μόνο την ηλιακή ομάδα, βρίσκοντας ηλικία, κανοντάς την int και περνόντας την απότ την συνάρτηση age_ranges, ως κλειδί και τον αριθμό 1 ως value
* κάνουμε reduce by key παίρνοντας το άθροισμα των επιμέρους τιμών
* ταξινομούμε κατα φθίνουσα σειρά

In [2]:
from pyspark.sql import SparkSession

sc = SparkSession \
    .builder \
    .appName("wordcount example") \
    .getOrCreate() \
    .sparkContext

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2015,application_1732639283265_1975,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Spark RDD code

# To log our application's execution time:
import time
import csv
from io import StringIO

def age_ranges(age: int):
    if age < 18:
        return "Children"
    elif age < 25:
        return "Young Adults"
    elif age < 65:
        return "Adults"
    else:
        return "Elderly"

def parse_csv(line):
    return next(csv.reader(StringIO(line)))
    
crime_2019_path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
crime_2020_path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"


start_time = time.time()

first_csv = sc.textFile(crime_2019_path)
second_csv = sc.textFile(crime_2020_path)

something = first_csv.union(second_csv) \
            .map(parse_csv) \
            .filter(lambda f: "aggravated assault" in f[9].lower()) \
            .map(lambda f: (age_ranges(int(f[11])), 1)) \
            .reduceByKey(lambda x,y: x+y) \
            .sortBy(lambda f: f[1], ascending=False)



print(something.collect())

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('Adults', 121093), ('Young Adults', 33605), ('Children', 15928), ('Elderly', 5985)]
Time taken: 19.45 seconds

In [4]:
from pyspark.sql import SparkSession

sc = SparkSession \
    .builder \
    .appName("wordcount example") \
    .getOrCreate() \
    .sparkContext

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Spark DataFrame code
from pyspark.sql.functions import col, lower, udf
from pyspark.sql.types import StringType
import time

def age_ranges(age: str):
    age = int(age)
    if age < 18:
        return "Children"
    elif age < 25:
        return "Young Adults"
    elif age < 65:
        return "Adults"
    else:
        return "Elderly"

crime_2019_path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2010_to_2019_20241101.csv"
crime_2020_path = "s3://initial-notebook-data-bucket-dblab-905418150721/CrimeData/Crime_Data_from_2020_to_Present_20241101.csv"



print("Starting")

start_time = time.time()

age_ranges_udf = udf(age_ranges, StringType())

crime_2019 = spark.read.csv(crime_2019_path, header=True)
crime_2020 = spark.read.csv(crime_2020_path, header=True)

something = crime_2019.union(crime_2020)
something = something.filter(lower(col("Crm Cd Desc")).contains("aggravated assault"))
something = something.withColumn("Age Range", age_ranges_udf(col("Vict Age")))
something = something.select("Age Range")
something = something.groupBy("Age Range").count()
something = something.orderBy("count", ascending=False)
something.show()

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Time taken: {elapsed_time:.2f} seconds")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Starting
+------------+------+
|   Age Range| count|
+------------+------+
|      Adults|121093|
|Young Adults| 33605|
|    Children| 15928|
|     Elderly|  5985|
+------------+------+

Time taken: 20.84 seconds